In [25]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: awml01
Azure region: uksouth
Subscription id: dabe5329-2380-4e80-a0cb-c9b370668176
Resource group: nanodegree01


In [28]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "awcluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
  cluster = ComputeTarget(workspace=ws, name=cluster_name)
  print('Found existing cluster, use it.')
except ComputeTargetException:
  compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', max_nodes=4)
  cluster = ComputeTarget.create(ws, cluster_name, compute_config)
  cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [29]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.001, 100.0),
        "--max_iter": choice(10, 50, 100, 500, 1000)
    }
)

# Specify a Policy
policy = BanditPolicy (
  slack_factor = 0.1,
  evaluation_interval = 1
)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=cluster,
    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=32,
    max_concurrent_runs=4,
    policy=policy,
    run_config=src)

In [30]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run_instance = exp.submit(config=hyperdrive_run_config)
RunDetails(run_instance).show()
run_instance.wait_for_completion(show_output = True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d
Web View: https://ml.azure.com/runs/HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d?wsid=/subscriptions/dabe5329-2380-4e80-a0cb-c9b370668176/resourcegroups/nanodegree01/workspaces/awml01&tid=6c3b75bb-be53-4e19-9a7d-f523c7e10636

Streaming azureml-logs/hyperdrive.txt

[2024-11-13T15:00:21.3791868Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-11-13T15:00:21.6245368Z][SCHEDULER][INFO]Scheduling job, id='HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_0' 
[2024-11-13T15:00:21.7650310Z][SCHEDULER][INFO]Scheduling job, id='HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_1' 
[2024-11-13T15:00:21.7663296Z][SCHEDULER][INFO]Scheduling job, id='HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_3' 
[2024-11-13T15:00:21.7675934Z][SCHEDULER][INFO]Scheduling job, id='HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_2' 
[2024-11-13T15:00:22.3568663Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_3' 
[2024-11-13T15:00:22.4031247Z][SCHEDULER][

{'runId': 'HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d',
 'target': 'awcluster',
 'status': 'Completed',
 'startTimeUtc': '2024-11-13T15:00:19.960315Z',
 'endTimeUtc': '2024-11-13T15:17:29.655236Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '512e3a7d-e8d8-4968-98ed-1ed5dbfbb249',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_16',
  'score': '0.9162367223065252',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_74d7fe5b-6c45-4fb7-ac25-82fa41baa92d_16'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-t

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [31]:
# Get your best run and save the model from that run.
best_run = run_instance.get_best_run_by_primary_metric()
best_run.download_file('outputs/regression_model.joblib')


In [32]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [33]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [34]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target = cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [35]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config)
RunDetails(automl_run).show()

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_9b7e04d9-5abe-41d7-8da9-cad14f8fc594,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"



In [36]:
import joblib
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run_job, best_automl_model = automl_run.get_output()
joblib.dump(value=best_automl_model, filename='automl_model.joblib')

['automl_model.joblib']

In [37]:
# Delete cluster
cluster.delete()